In [1]:
#Import standard packages
import pandas as pd
import numpy as np
import xarray as xr
import math

import os
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from pyaldata import *

import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io
from scipy import stats
from sklearn.metrics import r2_score
import pickle
from tqdm import tqdm
import csv

#Import function to get the covariate matrix that includes spike history from previous bins
from Neural_Decoding.preprocessing_funcs import get_spikes_with_history

#Import metrics
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.metrics import get_rho

#Import hyperparameter optimization packages
try:
    from hyperopt import fmin, hp, Trials, tpe, STATUS_OK
except ImportError:
    print("\nWARNING: hyperopt package is not installed. You will be unable to use section 5.")
    pass

#Import decoder functions
from Neural_Decoding.decoders import LSTMDecoder
from Neural_Decoding.decoders import SVClassification

In [2]:
# Function that reassigns different angles into classes from 1 to 8
# going anticlockwise starting from +x direction
def determine_angle(angle):
    if angle == 0:
        return 1
    elif angle == math.pi/4:
        return 2
    elif angle == math.pi/2:
        return 3
    elif angle == 3*math.pi/4:
        return 4
    elif angle == math.pi:
        return 5
    elif angle == -3*math.pi/4:
        return 6
    elif angle == -math.pi/2:
        return 7
    elif angle == -math.pi/4:
        return 8

In [3]:
# Load data
data_dir = '../raw_data/'
fname = os.path.join(data_dir, "Chewie_CO_CS_2016-10-14.mat")

# load TrialData .mat file into a DataFrame
df = mat2dataframe(fname, shift_idx_fields=True)

# Keep only successful trials
df = select_trials(df, "result == 'R'")

In [4]:
## Classification preprocessing

# Preprocessing
# combine time bins into longer ones, e.g. group 3 time bins together
td_class = combine_time_bins(df, 3)

# Obtain only the interval between idx_target_on and idx_go_cue
td_class = restrict_to_interval(td_class, start_point_name='idx_target_on', end_point_name='idx_go_cue')

# Remove low-firing neurons
td_class = remove_low_firing_neurons(td_class, "M1_spikes",  5)
td_class = remove_low_firing_neurons(td_class, "PMd_spikes", 5)

# total number of trials
N = td_class.shape[0]

#Number of M1_neurons
N_M1 = td_class.M1_spikes[0].shape[1]
#Number of PMd_neurons
N_PMd = td_class.PMd_spikes[0].shape[1]

M1_spikes = np.empty([N_M1,N])
PMd_spikes = np.empty([N_PMd,N])
y = np.empty([N,1])

for i in range(N):
    # Get the neuron spikes for a given trial in train data
    M1_trial = np.transpose(td_class.M1_spikes[i])
    PMd_trial = np.transpose(td_class.PMd_spikes[i])
    
    # Sum all the spikes in the given trial and save them
    M1_spikes[:,i] = np.sum(M1_trial, axis=1)
    PMd_spikes[:,i] = np.sum(PMd_trial, axis=1)
    
    # Get the label
    y[i] = determine_angle(td_class.target_direction[i])
    
# Build a feature vector
F_M1 = np.empty([N, N_M1])
F_PMd = np.empty([N, N_PMd])
for i in range(N):#in range(M1_spikes.shape[1]):
    total_M1_spikes = np.sum(M1_spikes[:,i]);
    total_PMd_spikes = np.sum(PMd_spikes[:,i])
    
    f_M1 = np.transpose(M1_spikes[:,i])/total_M1_spikes
    f_PMd = np.transpose(PMd_spikes[:,i])/total_PMd_spikes
    
    # Store average firing rates
    F_M1[i,:] = f_M1
    F_PMd[i,:] = f_PMd
    
# Combine M1 and PMd features
F_M1_PMd = np.concatenate((F_M1, F_PMd), axis = 1)

c:\users\jon\documents\biomedical engineering - ic\4. letnik\final year project\python\pyaldata\pyaldata\tools.py:979: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn("Assuming spikes are actually spikes and dividing by bin size.")


In [5]:
# Split the data into test and train subsets
split = int(0.8*N)

y_train = y[0:split-1]
y_test = y[split:]

F_M1_PMd_train = F_M1_PMd[0:split-1,:]
F_M1_PMd_test = F_M1_PMd[split:,:]


## Train classifiers
# Support vector classification
sv_classifier = SVClassification()

sv_classifier.fit(F_M1_PMd_train, np.squeeze(y_train))

In [6]:
## Regression preprocessing
folder='../preprocessed_data/'
#ENTER THE FOLDER THAT YOUR DATA IS IN

with open(folder+'individual_tar_data.pickle','rb') as f:
    M1, M1_PMd,pos_binned,vels_binned,sizes,trial_len=pickle.load(f,encoding='latin1') #If using python 3

neural_data = M1
kinematics = [pos_binned, vels_binned] 

LSTM_models = []
label_means = []
x_flat_mean = []
x_flat_std = []
N_tar = 8
for (idx,output) in enumerate(kinematics):
    start = 0
    end = sum(trial_len[0:sizes[0]])

    # Loop over the data to obtain decoders for all 8 targets
    for tar in tqdm(range(1, N_tar+1)): 

        # Preprocess data
        bins_before=6 #How many bins of neural data prior to the output are used for decoding
        bins_current=1 #Whether to use concurrent time bin of neural data
        bins_after=0 #How many bins of neural data after the output are used for decoding

        # Format for recurrent neural networks (SimpleRNN, GRU, LSTM)
        # Function to get the covariate matrix that includes spike history from previous bins
        X=get_spikes_with_history(neural_data[start:end],bins_before,bins_after,bins_current)

        # Format for Wiener Filter, Wiener Cascade, XGBoost, and Dense Neural Network
        #Put in "flat" format, so each "neuron / time" is a single feature
        X_flat=X.reshape(X.shape[0],(X.shape[1]*X.shape[2]))

        # Output covariates
        #Set decoding output
        y=output[start:end]

        # Split into training / testing / validation sets
        #Set what part of data should be part of the training/testing/validation sets
        training_range=[0, 0.7]
        testing_range=[0.7, 0.85]
        valid_range=[0.85,1]

        # Split data:
        num_examples=X.shape[0]

        #Note that each range has a buffer of"bins_before" bins at the beginning, and "bins_after" bins at the end
        #This makes it so that the different sets don't include overlapping neural data
        training_set=np.arange(int(np.round(training_range[0]*num_examples))+bins_before,int(np.round(training_range[1]*num_examples))-bins_after)
        testing_set=np.arange(int(np.round(testing_range[0]*num_examples))+bins_before,int(np.round(testing_range[1]*num_examples))-bins_after)
        valid_set=np.arange(int(np.round(valid_range[0]*num_examples))+bins_before,int(np.round(valid_range[1]*num_examples))-bins_after)

        #Get training data
        X_train=X[training_set,:,:]
        X_flat_train=X_flat[training_set,:]
        y_train=y[training_set,:]

        #Get testing data
        X_test=X[testing_set,:,:]
        X_flat_test=X_flat[testing_set,:]
        y_test=y[testing_set,:]

        #Get validation data
        X_valid=X[valid_set,:,:]
        X_flat_valid=X_flat[valid_set,:]
        y_valid=y[valid_set,:]

        # Process covariates
        #Z-score "X" inputs. 
       # X_train_mean=np.nanmean(X_train,axis=0)
       # X_train_std=np.nanstd(X_train,axis=0)
       # X_train=(X_train-X_train_mean)/X_train_std
        #X_test=(X_test-X_train_mean)/X_train_std
       # X_valid=(X_valid-X_train_mean)/X_train_std

        #Z-score "X_flat" inputs. 
        #X_flat_train_mean=np.nanmean(X_flat_train,axis=0)
       # X_flat_train_std=np.nanstd(X_flat_train,axis=0)
        #X_flat_train=(X_flat_train-X_flat_train_mean)/X_flat_train_std
        #X_flat_test=(X_flat_test-X_flat_train_mean)/X_flat_train_std
        #X_flat_valid=(X_flat_valid-X_flat_train_mean)/X_flat_train_std
        
        
        #Zero-center outputs
       # y_train_mean=np.mean(y_train,axis=0)
       # y_train=y_train-y_train_mean
        #y_test=y_test-y_train_mean
       # y_valid=y_valid-y_train_mean

       
        
        #Declare model
        model_lstm=LSTMDecoder(units=470,dropout=0.13857,num_epochs=7)

        #Fit model
        model_lstm.fit(X_train,y_train)

       

        #R2_vw = r2_score(y_valid,y_valid_predicted_wf, multioutput='variance_weighted')

        # Save the R2 value for a given neural data and kinematics
        #R2[idx,tar-1] = R2_vw
        LSTM_models.append(model_lstm)
        #label_means.append(y_train_mean)
        #x_flat_mean.append(X_flat_train_mean)
        #x_flat_std.append(X_flat_train_std)
        
        # Find new indexes based on trial_len and sizes variables
        start = end
        new_elements = sum(trial_len[sum(sizes[0:tar]):sum(sizes[0:tar+1])])
        end = end + new_elements

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [12:13<00:00, 91.71s/it]


In [7]:
# Import the whole data set
# combine time bins into longer ones
td_full = combine_time_bins(df, 3)

# Remove low-firing neurons
td_full = remove_low_firing_neurons(td_full, "M1_spikes",  5)
td_full = remove_low_firing_neurons(td_full, "PMd_spikes", 5)

# Get the signal from idx_go_cue
df.idx_movement_on = df.idx_movement_on.astype(int)
td_full = restrict_to_interval(td_full, start_point_name='idx_go_cue', end_point_name='idx_trial_end')


td_full = smooth_signals(td_full, ["M1_spikes", "PMd_spikes"], std=0.05)
 

c:\users\jon\documents\biomedical engineering - ic\4. letnik\final year project\python\pyaldata\pyaldata\tools.py:979: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn("Assuming spikes are actually spikes and dividing by bin size.")


In [8]:
## Classification preprocessing

# Preprocessing
# combine time bins into longer ones, e.g. group 3 time bins together
td_class = combine_time_bins(df, 3)

# Obtain only the interval between idx_target_on and idx_go_cue
td_class = restrict_to_interval(td_class, start_point_name='idx_target_on', end_point_name='idx_go_cue')

# Remove low-firing neurons
td_class = remove_low_firing_neurons(td_class, "M1_spikes",  5)
td_class = remove_low_firing_neurons(td_class, "PMd_spikes", 5)

# total number of trials
N = td_class.shape[0]

#Number of M1_neurons
N_M1 = td_class.M1_spikes[0].shape[1]
#Number of PMd_neurons
N_PMd = td_class.PMd_spikes[0].shape[1]

M1_spikes = np.empty([N_M1,N])
PMd_spikes = np.empty([N_PMd,N])
y = np.empty([N,1])

for i in range(N):
    # Get the neuron spikes for a given trial in train data
    M1_trial = np.transpose(td_class.M1_spikes[i])
    PMd_trial = np.transpose(td_class.PMd_spikes[i])
    
    # Sum all the spikes in the given trial and save them
    M1_spikes[:,i] = np.sum(M1_trial, axis=1)
    PMd_spikes[:,i] = np.sum(PMd_trial, axis=1)
    
    # Get the label
    y[i] = determine_angle(td_class.target_direction[i])
    
# Build a feature vector
F_M1 = np.empty([N, N_M1])
F_PMd = np.empty([N, N_PMd])
for i in range(N):#in range(M1_spikes.shape[1]):
    total_M1_spikes = np.sum(M1_spikes[:,i]);
    total_PMd_spikes = np.sum(PMd_spikes[:,i])
    
    f_M1 = np.transpose(M1_spikes[:,i])/total_M1_spikes
    f_PMd = np.transpose(PMd_spikes[:,i])/total_PMd_spikes
    
    # Store average firing rates
    F_M1[i,:] = f_M1
    F_PMd[i,:] = f_PMd
    
# Combine M1 and PMd features
F_M1_PMd = np.concatenate((F_M1, F_PMd), axis = 1)

# Split the data into test and train subsets
split = int(0.8*N)

y_train = y[0:split-1]
y_test = y[split:]

F_M1_PMd_train = F_M1_PMd[0:split-1,:]
F_M1_PMd_test = F_M1_PMd[split:,:]

In [9]:
## Make predictions
N_trials = 740
start = 0
kinematics = [td_full.pos, td_full.vel]

for (idx,output) in enumerate(kinematics):
    predictions = []
    y_valid_full = []
    y_pred_full = []
    for i in range(N_trials-int(0.8*N_trials)):
        #end = start + trial - 1
        #print(int(0.8*N_trials),i)
        neural_data = td_full.M1_spikes[int(0.8*N_trials)+i]
        y_valid = output[int(0.8*N_trials)+i]

        class_prediction = y_test[i]
        #class_prediction =sv_classifier.predict([F_M1_PMd_test[i]])
        #print(class_prediction)
        predictions.append(class_prediction)



        # Preprocess data
        bins_before=6 #How many bins of neural data prior to the output are used for decoding
        bins_current=1 #Whether to use concurrent time bin of neural data
        bins_after=0 #How many bins of neural data after the output are used for decoding

        # Format for recurrent neural networks (SimpleRNN, GRU, LSTM)
        # Function to get the covariate matrix that includes spike history from previous bins
        X=get_spikes_with_history(neural_data,bins_before,bins_after,bins_current)

        # Format for Wiener Filter, Wiener Cascade, XGBoost, and Dense Neural Network
        #Put in "flat" format, so each "neuron / time" is a single feature
        X_flat_final=X.reshape(X.shape[0],(X.shape[1]*X.shape[2]))


        X_flat_final = np.nan_to_num(X_flat_final)

       # X_flat_final = (X_flat_final-x_flat_mean[int(class_prediction-1)])/x_flat_std[int(class_prediction-1)]

       # y_valid = y_valid-label_means[int(class_prediction-1)]

        # Avoid some errors
        if X_flat_final.shape[0] == 0:
            continue
        
        X = np.nan_to_num(X)
        y_valid_predicted = LSTM_models[int(class_prediction-1)+8*idx].predict(X)


        y_valid_full.append(y_valid)
        y_pred_full.append(y_valid_predicted)

        # Update the starting point of the data for next iteration of the loop
        start = end + 1

        #y_pred_plot = y_valid_predicted
        #plt.plot(np.transpose(y_pred_plot[:,0]), np.transpose(y_pred_plot[:,1]))
    
    y_valid_full = np.array(y_valid_full)
    y_pred_full = np.array(y_pred_full)


    for i in range(y_valid_full.shape[0]):
        if i == 0:
            y_val = np.array(np.squeeze(y_valid_full[i]))
            y_pred = np.array(np.squeeze(y_pred_full[i]))
        else:
            y_val = np.concatenate((y_val, np.squeeze(y_valid_full[i])), axis=0)
            y_pred = np.concatenate((y_pred, np.squeeze(y_pred_full[i])), axis=0)

    R2_vw = r2_score(y_val,y_pred, multioutput='variance_weighted')
    print('R2 value:', R2_vw)

c:\users\jon\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
c:\users\jon\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:61: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


R2 value: -2.8809298297008303
R2 value: 0.9247391149016264


c:\users\jon\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
c:\users\jon\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:61: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
